In [1]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import time
import playsound
import speech_recognition as sr
from gtts import gTTS
import logging
from rasa_nlu import config
from rasa_nlu.model import Interpreter
import speech_recognition as sr
import datetime
from datetime import timedelta
import datefinder
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from dateutil.parser import parse
from tzlocal import get_localzone
from dateutil.relativedelta import relativedelta

logfile = 'nlu_model.log'
SCOPES = ['https://www.googleapis.com/auth/calendar']

CREDENTIALS_FILE = './credentials.json'

In [2]:
utter_greet=["Hey, how are you?","Hello, How are you doing?"] 
utter_reply=["I'm doing great. Please let me know what I can do for you.","I'm doing great. Tell me How can I help you today?","Good! Tell me what I can do for you today?","Good! How can I be of your service?"]
utter_help=["How can I help you?","Tell me How can I help you?","Tell me what all news you would like to get."]
utter_anything_else=["Is there anything else I can help you with?","Let me know if there is anything else I can help you with","Anything else?"]
utter_ofc=["Let me see what I can do.","Surely, I can help you."]
utter_bye=["Bye and have a nice day","Okay!Bye bye and have a nice day","Okay then have a great day!"]
utter_default=["I am not sure what you're aiming for.Please try again","I am sorry but I am not able to get you.Please try again","My apologies but I am not able to get you. Please try again"]
utter_confirm_created=["Event successfully created!","Your schedule has successfully been created!","The event has been created successfully!"]
utter_confirm_updated=["Your schedule has successfully been updated!","appointment updated!"]
utter_confirm_list=[ "Here is the list of your events!","here is your schedule !"]
utter_confirm_delete=["Event successfully deleted!","The appointment has been cancelled!"]
utter_conflict= ["There seems to be a conflict in your schedules.Please try another date."
    "Your schedules are clashing. Can you try another date?"]
utter_req_loc=["Where do you want to meet?","Do you have a specific location in your mind?"]
utter_req_time=["At what time should I schedule the meeting?","Please specify the time for meeting."]
    
utter_req_duration=["How long do you want the meeting for?","How long will the meeting last?"]
utter_Ok_wait =["Okay. Just wait for a moment"," Okay!"]
utter_welcome = ["Your welcome","you are most welcome","my pleasure"]

In [3]:
def speak(text):
    tts = gTTS(text=text, lang="en")
    filename = "voice.mp3"
    tts.save(filename)
    playsound.playsound(filename)

In [4]:
def recognize_speech_from_mic(recognizer, microphone):
    
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source) 
        audio = recognizer.listen(source)
        
    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        response["success"] = False
        response["error"] = "API unavailable/unresponsive"
    except sr.UnknownValueError:
        response["error"] = "Unable to recognize speech"

    return response



In [5]:
def run_nlu(nlu_path,query):
    logging.basicConfig(filename=logfile, level=logging.DEBUG)
    interpreter = Interpreter.load(nlu_path)
    ans = interpreter.parse(query)
    #print(ans['intent']['name'])
    event_details = {'summary':"event",'location':'Hyderabad' ,'date':"",'start_time':"",'end_time':"",'names':[] ,'description':"event",'duration':"",'new_time':"",'old_time':""}
    ran = ans['entities']
    for r in ran:
       # print(r['value'],r['entity'])
        if r['entity']=='email':
            event_details['names'].append(r['value'])
        else:
            event_details[r['entity']]= r['value']
    if event_details['summary']=="event" and event_details['description']!="event":
        event_details['summary']=event_details['description']
    elif event_details['summary']!="event" and event_details['description']=="event":
        event_details['description']=event_details['summary']
    return [ans['intent']['name'],event_details]

In [6]:
def get_calendar_service():
   creds = None
   # The file token.pickle stores the user's access and refresh tokens, and is
   # created automatically when the authorization flow completes for the first
   # time.
   if os.path.exists('token.pickle'):
       with open('token.pickle', 'rb') as token:
           creds = pickle.load(token)
   # If there are no (valid) credentials available, let the user log in.
   if not creds or not creds.valid:
       if creds and creds.expired and creds.refresh_token:
           creds.refresh(Request())
       else:
           flow = InstalledAppFlow.from_client_secrets_file(
               CREDENTIALS_FILE, SCOPES)
           creds = flow.run_local_server(port=0)

       # Save the credentials for the next run
       with open('token.pickle', 'wb') as token:
           pickle.dump(creds, token)

   service = build('calendar', 'v3', credentials=creds)
   return service

In [7]:
def get_date(query):
    date = parse(query, fuzzy_with_tokens=True)[0]
    return date
def get_local_zone():
    return str(get_localzone())

In [8]:
def get_email(name):
    import json
    db = open("db.json", 'r')
    email_ids = json.load(db)
    db.close()
    if name in email_ids:
        return email_ids[name]
    else:
        print("This name is not in the database.Can you please enter the email address of ",name)
        email_ids[name] = input()
    db = open('db.json',"w")
    json.dump(email_ids,db)
    db.close()

In [9]:
def get_duration(date):
    if date =="today":
        start_time = datetime.datetime.today().strftime('%Y-%m-%d')
        end_time =(datetime.datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
    elif date == "tomorrow":
        start_time =(datetime.datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
        end_time =(datetime.datetime.today() + timedelta(days=2)).strftime('%Y-%m-%d')
    elif date =="this week":
        start_time = datetime.datetime.today().strftime('%Y-%m-%d')
        end_time =(datetime.datetime.today() + timedelta(days=7)).strftime('%Y-%m-%d')
    elif date == "next week":
        start_time =(datetime.datetime.today() + timedelta(days=7)).strftime('%Y-%m-%d')
        end_time =(datetime.datetime.today() + timedelta(days=14)).strftime('%Y-%m-%d')
    elif date == "this month":
        start_time = datetime.datetime.today().strftime('%Y-%m-%d')
        end_time = (datetime.datetime.today()+ relativedelta(day=31)).strftime('%Y-%m-%d')   
    elif date == "next month":
        start_time = (datetime.datetime.today()+ relativedelta(day=31)+timedelta(days=1)).strftime('%Y-%m-%d')
        end_time = (datetime.datetime.today()+ relativedelta(day=31)+timedelta(days=1)+ relativedelta(day=31)).strftime('%Y-%m-%d')
    else:
        start_time = get_date(date).strftime('%Y-%m-%d')
        end_time = (get_date(date)+timedelta(days=1)).strftime('%Y-%m-%d')
    return [start_time,end_time]   
        

In [10]:
def get_event_id(start_time,end_time):
    service = get_calendar_service()
    #start_time = get_date("20th march 8 am").isoformat()+'Z'
    #end_time = get_date("20th april").isoformat()+'Z'
    events_result = service.events().list(calendarId='primary', timeMin=start_time, timeMax=end_time,
                                        singleEvents=True,
                                        orderBy='startTime').execute()
    events = events_result.get('items', [])
    ids = []
    if not events:
        return 0
    else:
        for i in range(0,len(events)):
            #print(events[i]['id'])
            ids.append(events[i]['id'])
    return ids

In [25]:
def find_conflicts(event_details):
    service = get_calendar_service()
    if event_details['date']!= '':
        duration = get_duration(event_details['date'])
    elif event_details['new_date']!= '' and event_details['date']== '':
        duration = get_duration(event_details['new_date'])
    else:
        print("Sorry but the system could not identify the date")
        speak("Sorry but the system could not identify the date")
    start_time = get_date(duration[0]).isoformat()+'Z'
    end_time = get_date(duration[1]).isoformat()+'Z'
    utc = get_local_zone()
    attendees = []
    names = event_details['names']
    for n in event_details['names']:
        attendees.append(get_email(n))
    #print(attendees)
    
    #start_time = get_date("20th march 8 am").isoformat()+'Z'
    #end_time = get_date("22th april").isoformat()+'Z'
    #attendees = ['carolnavya1999@gmail.com','carolnavya81@gmail.com']
    cal_list = []
    for a in attendees:
        cal_list.append({"id":str(a)})
    body = {
      "timeMin": start_time,
      "timeMax": end_time,
      "timeZone": 'Asia/Kolkata',
      "items": cal_list
    }
    conflict_count = {}
    eventsResult = service.freebusy().query(body=body).execute()
    cal_dict = eventsResult[u'calendars']
    #print(cal_dict['carolnavya81@gmail.com'])
    for c in cal_dict:
        if cal_dict[c]['busy'] == []:
            return 0
        else:
            return 1

In [26]:
def get_event_list(event_details):
    service = get_calendar_service()
    if event_details['date']!= '':
        duration = get_duration(event_details['date'])
    elif event_details['new_date']!= '' and event_details['date']== '':
        duration = get_duration(event_details['new_date'])
    else:
        print("Sorry but the system could not identify the date")
        speak("Sorry but the system could not identify the date")
    start_time = get_date(duration[0]).isoformat()+'Z'
    end_time = get_date(duration[1]).isoformat()+'Z'
    utc = get_local_zone()
    events_result = service.events().list(calendarId='primary', timeMin=start_time, timeMax=end_time,
                                        singleEvents=True,
                                        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')
        return
    print("%-10s\t%20s %30s "%('time','summary','status'))
    for event in events:
       # print(event,"\n")
        start = event['start'].get('dateTime', event['start'].get('date'))
        start = get_date(start).strftime('%d-%m-%Y at %H:%M')
        print("%-10s\t%-20s %25s " %(start,event['summary'],event['status']))
        #print(start,"\t",event['summary'],"\t\t",event['status'])
    return

In [27]:
def add_event(event_details):
    #print(event_details)
    service = get_calendar_service()
    summary = event_details['summary']
    location = event_details['location']
    if event_details['start_time']=="":
        print("Please enter the starting time for meeting")
        event_details['start_time']=str(input())
        #print(event_details['start_time'])
    start_time = get_date(str(event_details['date']+" "+event_details['start_time'])).strftime("%Y-%m-%dT%H:%M:%S")
    end_time = event_details['end_time']
    if end_time =="" and event_details['duration']!= "":
        end_time = get_date(str(event_details['date']+" "+event_details['start_time']+" "+event_details['duration'])).strftime("%Y-%m-%dT%H:%M:%S")
    elif end_time!="":
        end_time = get_date(str(event_details['date']+" "+event_details['end_time'])).strftime("%Y-%m-%dT%H:%M:%S")
    else:
         end_time = get_date(str(event_details['date']+" "+event_details['start_time']+" "+"30 minutes")).strftime("%Y-%m-%dT%H:%M:%S")
    timezone = get_local_zone()
    attendees = []
    names = event_details['names']
    for n in event_details['names']:
        attendees.append(get_email(n))
    #print(attendees)
    description = event_details['description']
    event = {
        'summary': summary,
        'location': location,
        'description': description,
        'start': {
            'dateTime': start_time,
            'timeZone': timezone,
        },
        'end': {
            'dateTime': end_time,
            'timeZone': timezone,
        },
        'attendees': [
        {'email':attendees },
    ],
        'reminders': {
            'useDefault': False,
            'overrides': [
                {'method': 'email', 'minutes': 24 * 60},
                {'method': 'popup', 'minutes': 10},
            ],
        },
    }
    x = find_conflicts(event_details)
    if x < 1:
        #print("GOES")
        return service.events().insert(calendarId='primary', body=event,sendNotifications=True).execute()
    else:
        return -1

In [28]:
def update_event():
    service = get_calendar_service()
    start_time = get_date("20th feb 8 am").isoformat()+'Z'
    end_time = get_date("20th march").isoformat()+'Z'
    ids = get_event_id()
    print(ids)
    if ids == 0:
        print("No event found to update")
    else:
        for i in ids:
            event = service.events().get(calendarId='primary', eventId=i).execute()
            event['summary'] = 'Appointment at Somewhere'
            updated_event = service.events().update(calendarId='primary', eventId=event['id'], body=event).execute()
            # Print the updated date.
            print(updated_event['updated'])

In [29]:
def cancel_event(event_details):
    service = get_calendar_service()
    if event_details['start_time']=="":
        duration = get_duration(event_details['date'])
        start_time = get_date(duration[0]).isoformat()+'Z'
        end_time = get_date(duration[1]).isoformat()+'Z'
    else:
        start_time = get_date(str(event_details['date']+" "+event_details['start_time'])).isoformat()+'Z'
        end_time = (start_time+timedelta(hours = 23)).isoformat()+'Z'                  
    #print(find_conflicts(start_time,end_time))
    ids = get_event_id(start_time,end_time)
    #print(ids)
    if ids == 0:
        print("No event found to delete")
    else:
        for i in ids:
            print("Deleting events")
            service.events().delete(calendarId='primary', eventId=i).execute()
            print("COMPLETED")

In [32]:
import random
def driver():
        var = 1
        x = "Hi"
        print(x)
        speak(x)
        while var == 1:
            """recognizer = sr.Recognizer()
            mic = sr.Microphone(device_index=0)
            response = recognize_speech_from_mic(recognizer, mic)
            if response['transcription']!= None:
                print(response['transcription'])
            else:
                print("Sorry I was unable to hear you. Please try again")
                speak("Sorry I was unable to hear you. Please try again")"""
            #query = response['transcription']
            query = input()
            output = run_nlu('./models/current/nlu',query)
            intent = output[0]
            event_details = output[1]
            print(intent,event_details)

            try:
                if intent == 'greet':
                    x = random.choice(utter_greet) 
                    print(x)
                    speak(x)
                elif intent == 'fine_ask':
                    x = random.choice(utter_reply) 
                    print(x)
                    speak(x)
                elif intent == 'fine_normal':
                    x = random.choice(utter_help) 
                    print(x)
                    speak(x)
                elif intent == 'thanks':
                    x = random.choice(utter_welcome) 
                    print(x)
                    speak(x)
                elif intent =='create_event':
                    x = random.choice(utter_ofc) 
                    print(x)
                    speak(x)
                    print(add_event(event_details)
                    if(add_event(event_details)== -1):
                        print("Sorry but the event couldnot be added")
                        speak("Sorry but the event couldnot be added")
                    else:
                        x = randon.choice(utter_confirm_created)
                        print(x)
                        speak(x)
                        y = random.choice(utter_anything_else)
                        print(y)
                        speak(y)
                elif intent == 'list_events':
                    x = random.choice(utter_confirm_list) 
                    print(x)
                    speak(x)
                    get_event_list(event_details)
                    y = random.choice(utter_anything_else)
                    print(y)
                    speak(y)
                elif intent == 'delete_event':
                    x = random.choice(utter_ofc) 
                    print(x)
                    speak(x)
                    cancel_event(event_details)
                    x = random.choice(utter_confirm_delete) 
                    print(x)
                    speak(x)
                    y = random.choice(utter_anything_else)
                    print(y)
                    speak(y)
                elif intent == 'update_events':
                    x = random.choice(utter_ofc) 
                    print(x)
                    speak(x)
                    update_list(event_details)
                    x = random.choice(utter_confirm_updated) 
                    print(x)
                    speak(x)
                    y = random.choice(utter_anything_else)
                    print(y)
                    speak(y)
                elif intent== 'say_bye':
                    x = random.choice(utter_bye) 
                    print(x)
                    speak(x)
                    var = 0
            except:
                x = random.choice(utter_default) 
                print(x)
                speak(x)
                

SyntaxError: invalid syntax (<ipython-input-32-670f4df48e8f>, line 45)

In [29]:
driver()

Hi
None
hello
Hey, how are you?
None
show my schedule for this month
here is your schedule !
time				summary				status
2020-02-28T02:00:00+05:30 	 event 	 confirmed
None
Can you create a meeting with carol and anju on 8th march at 3am
Let me see what I can do.
This name is not in the database.Can you please enter the email address of  anju
anjaliraotamvada@gmail.com
Your schedule has successfully been created!
None
thanks
Your welcome
None
bye
Bye and have a nice day


In [30]:
driver()

Hi
Hemant
hi
Hey, how are you?
None
I am fine
Tell me How can I help you?
I would like to schedule a meeting
Schedule a meeting with carol on 3rd march at 4pm
Surely, I can help you.
Event successfully created!
None
Schedule a meeting with hari on 3rd march at 4pm
Surely, I can help you.
This name is not in the database.Can you please enter the email address of  hari
hari123@gmail.com
My apologies but I am not able to get you. Please try again
None
bye
Okay!Bubye and have a nice day


In [ ]:
driver()

Hi
ginti wala
hi
Hello, How are you doing?
how are you
I am fine
Tell me How can I help you?
Carol on 28 March
Schedule the meeting with carol on 28th march at 10 am
Let me see what I can do.
My apologies but I am not able to get you. Please try again
Sorry I was unable to hear you. Please try again
show my schedule for 28th march
Here is the list of your events!
I am sorry but I am not able to get you.Please try again
calculator
show my schedule for today
Here is the list of your events!
time      	             summary                         status 
11-03-2020 at 10:00	event                                confirmed 
Is there anything else I can help you with?
Sorry I was unable to hear you. Please try again


In [17]:
print("Can you update my meeting with anjali from march")
print("Surely I can help you!")
print("Event successfully deleted")

Can you update my meeting with anjali from march
Surely I can help you!
Event successfully deleted


In [43]:
import os

os.listdir()

['custom_actions-Copy1.ipynb',
 'voice.mp3',
 'custom_actions.ipynb',
 'nlu_config.yml',
 '.DS_Store',
 'python files test',
 'db.json',
 'nlu_model.ipynb',
 'errors.json',
 'Driver.ipynb',
 'actions.py',
 'models',
 '__pycache__',
 'endpoints.yml',
 'nlu_model.log',
 'results',
 '.ipynb_checkpoints',
 'domain.yml',
 'token.pickle',
 'client_secret_139139097066-psoe3evj2quq6nuhh6r7a6urjia2v1pt.apps.googleusercontent.com.json',
 'data',
 'notebook.tex',
 'cal.ipynb',
 'policy.yml']

In [ ]:
driver()

Hi
hi
greet {'summary': 'event', 'location': 'Hyderabad', 'date': '', 'start_time': '', 'end_time': '', 'names': [], 'description': 'event', 'duration': '', 'new_time': '', 'old_time': ''}
Hey, how are you?


In [24]:
driver()

Hi
hi
greet {'summary': 'event', 'location': 'Hyderabad', 'date': '', 'start_time': '', 'end_time': '', 'names': [], 'description': 'event', 'duration': '', 'new_time': '', 'old_time': ''}
Hello, How are you doing?
I am fine
fine_normal {'summary': 'event', 'location': 'Hyderabad', 'date': '', 'start_time': '', 'end_time': '', 'names': [], 'description': 'event', 'duration': '', 'new_time': '', 'old_time': ''}
Tell me How can I help you?
Can you schedule a meeting with carol on june 5
create_event {'summary': 'event', 'location': 'Hyderabad', 'date': 'june 5', 'start_time': '', 'end_time': '', 'names': ['carol'], 'description': 'event', 'duration': '', 'new_time': '', 'old_time': ''}
Let me see what I can do.
Please enter the starting time for meeting
2 am
My apologies but I am not able to get you. Please try again
bye
say_bye {'summary': 'event', 'location': 'Hyderabad', 'date': '', 'start_time': '', 'end_time': '', 'names': [], 'description': 'event', 'duration': '', 'new_time': '', 

In [19]:
print("Can you update my meeting from june 5 from 2pm to 4pm")
print("Surely, I can help you")
print("Event updated")

Can you update my meeting from june 5 from 2pm to 4pm
Surely, I can help you
Event updated
